In [14]:
import sys
import os
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.signal import find_peaks, welch
from vmdpy import VMD
from scipy.stats import pearsonr
from scipy.signal.windows import gaussian
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

In [15]:
window_size = 20
rPPG_fps = 30
ecg_fps = 2000
ecg_peak_height = 0.5
lowcut = 0.5
highcut = 3.0

In [16]:
# rPPG 处理函数

# 巴特沃斯滤波
def bandpass_filter(data, fs, lowcut, highcut):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    order = 3
    b, a = signal.butter(order, [low, high], btype='bandpass')
    filtered_signal = signal.filtfilt(b, a, data)
    return filtered_signal


# 合并曲线
def merge_curves(curves):
    n_samples = len(curves[0])
    merged_curve = np.zeros(n_samples)
    for curve in curves:
        merged_curve += curve   
    return merged_curve


# 平滑信号
# def rPPG_smooth(rPPGs):
def rPPG_smooth(signal, window_len=5, window='hanning'):
    if window_len > len(signal):
        return signal
    window = getattr(np, window)(window_len)
    smoothed_signal = np.convolve(signal, window, mode='same')[:len(signal)]
    return smoothed_signal


# 归一化
def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data


# 生成虚拟rPPG
def generate_vRPPG(rPPG_signal):
    rPPGs_smoothed = rPPG_smooth(rPPG_signal)
    # rPPGs_filter = bandpass_filter(rPPGs_smoothed, rPPG_fps, lowcut, highcut)
    # u, u_hat, omega = VMD(rPPGs_smoothed, 15, 0, 3, 0, 1, 0.0000001)
    # signal1 = np.hstack(u[0:1])
    # curves = [signal1]
    # merged_curve = merge_curves(curves)
    
    peaks, _ = find_peaks(rPPGs_smoothed, distance=lowcut*rPPG_fps)
    peak_indices = np.array(peaks)
    vRPPG_signal = np.zeros_like(rPPG_signal)
    num_peaks = len(peak_indices)
    # 处理第一个周期，确保从信号开始就有连续的余弦波形
    if num_peaks > 1:
        start = peak_indices[0]
        end = peak_indices[1]
        duration = end - start
        x = np.linspace(0, duration, num=end + 1)  # end+1 确保包含 end 索引
        vRPPG_signal[:end + 1] = np.cos(2 * np.pi * (x - 0) / duration)
    # 处理中间的周期
    for i in range(1, num_peaks - 1):
        start = peak_indices[i]
        end = peak_indices[i + 1]
        duration = end - start
        x = np.linspace(start, end, num=end - start + 1)
        vRPPG_signal[start:end + 1] = np.cos(2 * np.pi * (x - start) / duration)
    # 处理最后一个周期，确保余弦波形在信号末尾也是连续的
    if num_peaks > 1:
        start = peak_indices[-1]
        duration = len(rPPG_signal) - start
        x = np.linspace(0, duration, num=len(rPPG_signal) - start)
        vRPPG_signal[start:] = np.cos(2 * np.pi * (x - 0) / duration)
    return vRPPG_signal, rPPGs_smoothed

In [17]:
# ECG 处理函数

# 平滑信号
def ECG_smooth(signal, window_len=100, window='hanning'):
    if window_len > len(signal):
        return signal
    window = getattr(np, window)(window_len)
    smoothed_signal = np.convolve(signal, window, mode='same')[:len(signal)]
    return smoothed_signal

# 生成vECG信号
def generate_vECG(ecg_signal):
    ecg_signal_smoothed = ECG_smooth(ecg_signal)
    peaks, _ = find_peaks(ecg_signal_smoothed, height=ecg_peak_height, distance=ecg_fps*0.5)
    peak_indices = np.array(peaks)
    vECG_signal = np.zeros_like(ecg_signal)
    num_peaks = len(peak_indices)
    # 处理第一个周期，确保从信号开始就有连续的余弦波形
    if num_peaks > 1:
        start = peak_indices[0]
        end = peak_indices[1]
        duration = end - start
        x = np.linspace(0, duration, num=end + 1)  # end+1 确保包含 end 索引
        vECG_signal[:end + 1] = np.cos(2 * np.pi * (x - 0) / duration)
    # 处理中间的周期
    for i in range(1, num_peaks - 1):
        start = peak_indices[i]
        end = peak_indices[i + 1]
        duration = end - start
        x = np.linspace(start, end, num=end - start + 1)
        vECG_signal[start:end + 1] = np.cos(2 * np.pi * (x - start) / duration)
    # 处理最后一个周期，确保余弦波形在信号末尾也是连续的
    if num_peaks > 1:
        start = peak_indices[-1]
        duration = len(ecg_signal) - start
        x = np.linspace(0, duration, num=len(ecg_signal) - start)
        vECG_signal[start:] = np.cos(2 * np.pi * (x - 0) / duration)
    return vECG_signal

In [18]:
# 心率计算函数
def psd_hr(signal, fs, last_bpm):
    time = np.linspace(0, len(signal)/fs, len(signal), endpoint=False)
    peaks, _ = find_peaks(signal)  # 寻找波峰
    peak_times = time[peaks]  # 波峰的时间点
    peak_intervals = np.diff(peak_times)  # 波峰之间的时间间隔
    if peak_intervals.size > 0:
        frequency = 1 / peak_intervals.mean()  # 计算平均频率
    else:
        frequency = np.nan  # 没有足够的峰值来计算频率
    bpm = frequency * 60
    if last_bpm > 0:
        bpm = (last_bpm * 0.9) + (bpm * 0.1)
    return bpm

In [19]:
# 主函数

ROOT_PATH = r"F:\ZJXU-HR-V1-130"
index = 90
dir_path = rf"F:\ZJXU-HR-V1-130\s{index}"
# dir_path = rf"F:\ZJXU-HR-V1-130\s{index}_talk"

def cal_rPPG_ECG_hr(dir_path):
    folder_name = os.path.basename(dir_path)
    print(f"{folder_name} 开始处理")

    # rPPG
    rPPG_path = os.path.join(dir_path, r'rPPG_bigger.csv')
    rPPG_df = pd.read_csv(rPPG_path)
    R_col = rPPG_df['R'].values.flatten()
    G_col = rPPG_df['G'].values.flatten()
    B_col = rPPG_df['B'].values.flatten()

    rPPGs = G_col/R_col + G_col/B_col
    vRPPG, rPPGs_smoothed = generate_vRPPG(rPPGs)
    
    plt.plot(normalize(rPPGs), label='rPPGs')
    plt.plot(normalize(rPPGs_smoothed), label='rPPGs_smoothed')
    # plt.plot(normalize(rPPGs_filter), label='rPPGs_filter')
    # plt.plot(normalize(merged_curve), label='merged_curve')
    # plt.plot(normalize(vRPPG), label='vRPPG')
    # plt.plot(normalize(pos_rPPG), label='pos_rPPG')
    plt.legend()
    plt.show(block=True)

    rPPG_hr = []
    last_bpm = 0
    for i in range(0, len(vRPPG)-rPPG_fps*window_size, rPPG_fps*1):
        bpm = psd_hr(vRPPG[i:i+rPPG_fps*window_size], rPPG_fps, last_bpm)
        rPPG_hr.append(bpm)
        last_bpm = bpm

    # ECG
    ECG_path = os.path.join(dir_path, r'ECG_EEG_RSP.csv')
    ECG_df = pd.read_csv(ECG_path, header=0)[['ECG, X, RSPEC-R']]
    # [['ECG B, X, ECG2-R']]
    ECGs = ECG_df.values.flatten()

    vECG = generate_vECG(ECGs).flatten()

    ECG_hr = []
    last_bpm = 0
    for i in range(0, len(vECG)-ecg_fps*window_size, ecg_fps*1):
        bpm = psd_hr(vECG[i:i+ecg_fps*window_size], ecg_fps, last_bpm)
        ECG_hr.append(bpm)
        last_bpm = bpm

    # pcc
    print("vRPPG bpm:", np.mean(rPPG_hr))
    print("ECG bpm:", np.mean(ECG_hr))
    min_num = min(len(rPPG_hr), len(ECG_hr))
    pcc, _ = pearsonr(rPPG_hr[:min_num], ECG_hr[:min_num])  
    print(f"pcc: {round(pcc,4)}")
    
    # 绘图
    plt.title(folder_name + f'  pcc={pcc}')
    plt.plot(rPPG_hr, label=f'rPPG_hr={round(np.mean(rPPG_hr), 2)}')
    plt.plot(ECG_hr, label=f'ECG_hr={round(np.mean(ECG_hr), 2)}')
    plt.legend()
    plt.show(block=True)


cal_rPPG_ECG_hr(dir_path)


s90 开始处理
vRPPG bpm: 85.17399743379836
ECG bpm: 84.47067475525206
pcc: 0.9098
